In [2]:
# -.-|m { input: false, output: false, input_fold: show}

import logging
from os import path, system

import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import session_info
import tomlkit
from anndata import AnnData
from pandas import DataFrame
from torch.cuda import is_available as cuda_is_available

from utils.util_funcs import cell_typist_annotate

In [14]:
# Add CELL_TYPIST model(s) to use
CELL_TYPIST_MODELS: list[str] = []

In [15]:
# | echo: false
# | output: false
# | warning: false

## Pipeline parameters
with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

In [16]:
ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
COUNTS_LAYER = config["normalization"]["COUNTS_LAYER"]
CLUSTERING_COL = config["clustering"]["CLUSTERING_COL"]
TISSUE = config["basic"]["TISSUE"]
ANNOTATION_METHOD = config["annotation"]["ANNOTATION_METHOD"]
NORMAMALIZATION_LAYER = config["normalization"]["NORMALIZATION_METHOD"]

In [19]:
adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))

In [20]:
# Getting a stable counts layer to be used later, setting X to be raw count values.
if COUNTS_LAYER == "X":
    adata.layers["counts"] = adata.X.copy()
    COUNTS_LAYER = "counts"
elif COUNTS_LAYER in adata.layers.keys():
    adata.X = adata.layers[COUNTS_LAYER].copy()
else:
    raise ValueError("{COUNTS_LAYER} layer can't be found in the object")

if ANNOTATION_METHOD == "celltypist":
    cell_typist_annotate(adata, CELL_TYPIST_MODELS)
if ANNOTATION_METHOD == "scGPT":
    if cuda_is_available():
        system(f"pixi run -e scgpt -i {path.join(DIR_SAVE, 'adata.h5ad')}")
    else:
        print("CUDA is not available, scGPT will not be run efficiently on CPU")
        exit(code=0)

if ANNOTATION_METHOD == "scTAB":
    system(f"pixi run -e sctab -i {path.join(DIR_SAVE, "adata.h5ad")}")
    adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))


Resources are ready
Loading genes from model


ModuleNotFoundError: No module named 'cellnet'

In [3]:
session_info.show()